In [4]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-r77emb1g
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-r77emb1g
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-4pmo6fhg
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-4pmo6fhg
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [ ]:
import clip
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [ ]:
# I refer https://github.com/openai/CLIP colab code.

In [ ]:
#!pip install transformers
#from transformers import CLIPProcessor, CLIPModel
#https://huggingface.co/openai/clip-vit-base-patch32

In [ ]:
model, preprocess = clip.load("ViT-B/32")
model.cuda().eval()

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [ ]:
import torch
from torchvision import transforms, datasets
import numpy as np

In [ ]:
preprocess

Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=warn)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x78e06ac49240>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [ ]:
test_dataset = datasets.CIFAR10(root="./data/",
                                train=False,
                                download=True,
                                transform=preprocess)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=2)


Files already downloaded and verified


In [ ]:
descriptions = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
text_descriptions = [f"This is a photo of a {label}" for label in descriptions]
text_tokens = clip.tokenize(text_descriptions).cuda()
with torch.no_grad():
  text_features = model.encode_text(text_tokens).float()
  print(text_features.shape)
  text_features /= text_features.norm(dim = 1, keepdim=True)
  print(text_features)

torch.Size([10, 512])
tensor([[ 0.0190,  0.0416, -0.0104,  ..., -0.0299,  0.0032, -0.0214],
        [ 0.0184,  0.0237, -0.0171,  ..., -0.0169, -0.0352, -0.0087],
        [ 0.0321,  0.0530, -0.0212,  ..., -0.0465,  0.0098, -0.0242],
        ...,
        [ 0.0241,  0.0200, -0.0277,  ..., -0.0442, -0.0127, -0.0004],
        [ 0.0261,  0.0195, -0.0201,  ..., -0.0488, -0.0012, -0.0326],
        [ 0.0225,  0.0464, -0.0160,  ..., -0.0414, -0.0162, -0.0054]],
       device='cuda:0')


In [ ]:
correct = 0
total = 0
try:
    for image, label in test_loader:
      image_features = model.encode_image(image.cuda()).float()
      image_features /= image_features.norm(dim = 1, keepdim=True)
      sim_vector = (image_features @ text_features.T)
      argmax = np.argmax(sim_vector.softmax(dim = 1).detach().cpu().numpy(), axis = 1)
      correct += np.sum(np.equal(label.numpy(), argmax))
      total += 2
except:
  pass
finally:
  accuracy = correct/total
  print(f"Breaking, accuracy : {accuracy}")


Breaking, accuracy : 0.9095238095238095


**Linear Probe**

refer: https://github.com/openai/CLIP

In [5]:
import clip
import torch

import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR100
from tqdm import tqdm

# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

# Load the dataset
root = ".data/"
train = CIFAR100(root, download=True, train=True, transform=preprocess)
test = CIFAR100(root, download=True, train=False, transform=preprocess)


def get_features(dataset):
    all_features = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=100)):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

# Calculate the image features
train_features, train_labels = get_features(train)
test_features, test_labels = get_features(test)

# Perform logistic regression
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)

# Evaluate using the logistic regression classifier
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 100/100 [00:22<00:00,  4.40it/s]
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy = 80.010
